In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

import torchvision
from torchvision import transforms
import torch
import PIL

In [ ]:
class dc_dataset(Dataset):
    def __init__(self, path ,transform=None):
        self.path = path
        self.img_list = os.listdir(path)
        self.transform = transform

    def __getitem__(self, idx):
        if self.img_list[idx][0] =='d':
            label = 0
        else:
            label = 1
        image = Image.open( os.path.join(self.path, self.img_list[idx]) ).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.img_list)

In [ ]:
from torchvision import transforms

tran = transforms.Compose([
    transforms.Resize((60, 60)),
    transforms.ToTensor()  # PIL → Tensor [C, H, W], 0~1 float32
])

train = dc_dataset('./train', transform=tran)
test = dc_dataset('./test', transform=tran)

In [ ]:
train_loader = DataLoader(dataset = train, batch_size = 64, shuffle = True, num_workers = 2)
test_loader = DataLoader(dataset = test, batch_size = 64, shuffle = True, num_workers = 2)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class cnn1(nn.Module):
    def __init__(self):
        super(cnn1, self).__init__()
        # 입력 채널: 3 (RGB), 출력 채널: 8, 커널 크기: 3x3
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        # 두 번째 합성곱 층: 입력 채널 8, 출력 채널 16
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        # 완전연결층: 16채널, 15x15 크기 (60x60 -> maxpool 2번으로 15x15)
        self.fc1 = nn.Linear(16 * 15 * 15, 2)  # 클래스 2개

    def forward(self, x):
        x = F.relu(self.conv1(x))      # [B, 3, 60, 60] -> [B, 8, 60, 60]
        x = F.max_pool2d(x, 2)         # [B, 8, 60, 60] -> [B, 8, 30, 30]
        x = F.relu(self.conv2(x))      # [B, 8, 30, 30] -> [B, 16, 30, 30]
        x = F.max_pool2d(x, 2)         # [B, 16, 30, 30] -> [B, 16, 15, 15]
        x = x.view(x.size(0), -1)      # [B, 16, 15, 15] -> [B, 16*15*15]
        x = self.fc1(x)                # [B, 16*15*15] -> [B, 2]
        return x

In [ ]:
# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델, 손실함수, 옵티마이저 정의
model = cnn1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5)  

epoch = 10
BATCH_SIZE = 64

In [ ]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()


        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))


In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0


    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(device)
            label = label.to(device)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()


    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5)  

In [19]:
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))


Train Epoch: 1 [0/25000 (0%)]	Train Loss: 0.445292
Train Epoch: 1 [12800/25000 (51%)]	Train Loss: 0.548132

[EPOCH: 1], 	Test Loss: 0.5173, 	Test Accuracy: 75.33 % 

Train Epoch: 2 [0/25000 (0%)]	Train Loss: 0.574407
Train Epoch: 2 [12800/25000 (51%)]	Train Loss: 0.469801

[EPOCH: 2], 	Test Loss: 0.5146, 	Test Accuracy: 75.48 % 

Train Epoch: 3 [0/25000 (0%)]	Train Loss: 0.492079
Train Epoch: 3 [12800/25000 (51%)]	Train Loss: 0.512940

[EPOCH: 3], 	Test Loss: 0.5134, 	Test Accuracy: 75.53 % 

Train Epoch: 4 [0/25000 (0%)]	Train Loss: 0.485815
Train Epoch: 4 [12800/25000 (51%)]	Train Loss: 0.634937

[EPOCH: 4], 	Test Loss: 0.5140, 	Test Accuracy: 74.94 % 

Train Epoch: 5 [0/25000 (0%)]	Train Loss: 0.456752
Train Epoch: 5 [12800/25000 (51%)]	Train Loss: 0.501680

[EPOCH: 5], 	Test Loss: 0.5104, 	Test Accuracy: 75.98 % 

Train Epoch: 6 [0/25000 (0%)]	Train Loss: 0.451191
Train Epoch: 6 [12800/25000 (51%)]	Train Loss: 0.477610

[EPOCH: 6], 	Test Loss: 0.5165, 	Test Accuracy: 74.59 % 

Trai